# GAN Generative Adversarial Network

- 서로 대립하는 두 신경망을 경쟁시켜 결과물을 생성하는 방법
- 위조지폐범과 경찰의 비유


1. 실제 이미지를 구분자(Disriminator)에게 이 이미지가 진짜인지 판단하게 함.
2. 생성자(Generator)를 통해 노이즈로 부터 임의의 이미지를 만들고 이것을 다시 구분자에게 판단하게 함.
이 둘을 반복하여 생성자는 구분자를 속여 최대한 진짜처럼 보이게 하고, 구분자는 최대한 진짜와 가짜를 구별하도록 훈련


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

/Users/yeomyeongwoo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 하이퍼 파라미터 따로 빼기
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128

In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [4]:
# 생성자 Generator
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
print(G_W1)
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

<tf.Variable 'Variable:0' shape=(128, 256) dtype=float32_ref>


In [5]:
# Discriminator
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [6]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [7]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [14]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [15]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [16]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

In [17]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [18]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [28]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    
    epoch_str = '{:04d}'.format(epoch + 1)
    loss_d = '{:.4f}'.format(loss_val_D)
    loss_g = '{:.4f}'.format(loss_val_G)
    print(f'Epoch: {epoch_str} D loss : {loss_d} G loss : {loss_g}')
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
    
print(complete)

Epoch: 0001 D loss : -0.5877 G loss : -2.0141
Epoch: 0002 D loss : -0.2752 G loss : -2.5837
Epoch: 0003 D loss : -0.1684 G loss : -3.0263
Epoch: 0004 D loss : -0.4188 G loss : -1.7185
Epoch: 0005 D loss : -0.2748 G loss : -2.0361
Epoch: 0006 D loss : -0.2785 G loss : -2.6209
Epoch: 0007 D loss : -0.1306 G loss : -3.0597
Epoch: 0008 D loss : -0.2035 G loss : -2.5873
Epoch: 0009 D loss : -0.3275 G loss : -2.8343
Epoch: 0010 D loss : -0.2203 G loss : -2.6602
Epoch: 0011 D loss : -0.2475 G loss : -2.9248
Epoch: 0012 D loss : -0.2566 G loss : -2.6139
Epoch: 0013 D loss : -0.2606 G loss : -2.7115
Epoch: 0014 D loss : -0.2235 G loss : -3.0715
Epoch: 0015 D loss : -0.2752 G loss : -2.9814
Epoch: 0016 D loss : -0.4009 G loss : -2.5729
Epoch: 0017 D loss : -0.3948 G loss : -2.9074
Epoch: 0018 D loss : -0.4249 G loss : -2.5780
Epoch: 0019 D loss : -0.4483 G loss : -2.3775
Epoch: 0020 D loss : -0.3129 G loss : -2.7017
Epoch: 0021 D loss : -0.3869 G loss : -2.7388
Epoch: 0022 D loss : -0.3643 G los

NameError: name 'complete' is not defined